In [2]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
cfns = pd.read_csv('../data/classifications_cset.csv')

In [4]:
cfns.sample(5)

,classifications.Annotator,classifications.Annotation Status,classifications.Reviewer,classifications.Quality Control,classifications.Full Description,classifications.Short Description,classifications.Beginning Date,classifications.Ending Date,classifications.Location,classifications.Near Miss,...,classifications.Sector of Deployment,classifications.Public Sector Deployment,classifications.Nature of End User,classifications.Level of Autonomy,classifications.Relevant AI functions,classifications.AI Techniques,classifications.AI Applications,classifications.Physical System,classifications.Problem Nature,classifications.Publish
71,2,3. In peer review,6,False,Facebook's automatic language translation soft...,Facebook's automatic language translation soft...,2017-10-15T07:00:00.000Z,2017-10-15T07:00:00.000Z,"Beitar Illit, Israel",Harm caused,...,Information and communication,False,Amateur,High,"[""Perception"",""Cognition"",""Action""]",Facebook language translation software,"[""language translation""]","[""Software only""]","[""Specification""]",False
41,5,6. Complete and final,5,False,"From 1982 to 1986, St George's Hospital Medica...","From 1982 to 1986, St George's Hospital Medica...",1982-01-01T00:00:00.000Z,1986-01-01T00:00:00.000Z,"London, England",Harm caused,...,Human health and social work activities,False,Amateur,Medium,"[""Cognition""]",Machine learning,"[""decision support""]","[""Software only""]","[""Specification""]",True
19,2,6. Complete and final,6,False,The Winograd Schema Challenge in 2016 highligh...,The 2016 Winograd Schema Challenge highlighted...,2016-01-01T00:00:00.000Z,2016-01-01T00:00:00.000Z,"New York, NY",Unclear/unknown,...,"Professional, scientific and technical activities",False,Expert,High,"[""Perception"",""Cognition"",""Action""]",NaN,"[""""]","[""Software only""]",[],True
81,6,6. Complete and final,5,False,"In October 2020, Facebook incorrectly labelled...",Facebook incorrectly labels content relating t...,2020-10-21T07:00:00.000Z,2020-10-21T07:00:00.000Z,"Lagos, Nigeria",Unclear/unknown,...,Information and communication,False,Amateur,Medium,"[""Perception"",""Cognition"",""Action""]",machine learning; supervised learning; open-so...,"[""content moderation"",""recommendation engine""]","[""Software only""]","[""Specification""]",True
68,2,6. Complete and final,2,False,A factory robot at the SKH Metals Factory in M...,A factory robot at the SKH Metals Factory in M...,2015-08-13T07:00:00.000Z,2015-08-13T07:00:00.000Z,"Manesar, India",Harm caused,...,NaN,False,NaN,NaN,[],NaN,"[""""]",[],[],True


In [47]:
# clean up!
cfns.columns = cfns.columns.str.lower().str.replace('classifications.', '').str.replace(' ', '_')
cfns['beginning_date'] = pd.to_datetime(cfns['beginning_date'], utc=True).dt.date
cfns['ending_date'] = pd.to_datetime(cfns['ending_date'], utc=True).dt.date

def parse_list_columns(c):
    if type(c[0]) == str and c[0].startswith('['):
        return c.apply(eval)
    else: return c

cfns = cfns.apply(parse_list_columns)

In [80]:
cfns.columns

Index(['annotator', 'annotation_status', 'reviewer', 'quality_control',
       'full_description', 'short_description', 'beginning_date',
       'ending_date', 'location', 'near_miss', 'named_entities',
       'technology_purveyor', 'intent', 'severity', 'lives_lost',
       'harm_distribution_basis', 'harm_type', 'infrastructure_sectors',
       'financial_cost', 'laws_implicated', 'ai_system_description',
       'data_inputs', 'system_developer', 'sector_of_deployment',
       'public_sector_deployment', 'nature_of_end_user', 'level_of_autonomy',
       'relevant_ai_functions', 'ai_techniques', 'ai_applications',
       'physical_system', 'problem_nature', 'publish'],
      dtype='object')

In [5]:
cfns.shape

(96, 33)

In [128]:
cfns[cfns['publish'] == True].shape

(63, 33)

In [129]:
# harms: multiple options
cfns['harm_type'].apply(pd.Series).stack().value_counts()

Harm to social or political systems                           18
Harm to physical health/safety                                17
Harm to civil liberties                                       17
Psychological harm                                            17
Financial harm                                                 9
Harm to physical property                                      8
Harm to intangible property                                    3
Other:Privacy                                                  2
Other:Reputational harm/social harm (libel and defamation)     1
Other:Reputational harm                                        1
Other:Unable to detect facial features                         1
Other:Reputational harm; False incarceration                   1
Other:Harm to publicly available information                   1
dtype: int64

In [131]:
cfns['technology_purveyor'].apply(pd.Series).stack().value_counts()

Google                                   18
Amazon                                    6
Facebook                                  4
Microsoft                                 4
Tesla                                     4
                                         ..
Delphi                                    1
Jordan Peele                              1
Centrelink Master Program                 1
St George’s Hospital Medical School       1
National Residential Matching Program     1
Length: 83, dtype: int64

In [9]:
cfns[cfns['classifications.Technology Purveyor'].str.contains('AI')]['classifications.Full Description'].values

array(['OpenAI published a post about its findings when using Universe, a software for measuring and training AI agents to conduct reinforcement learning experiments.Universe was used to train an AI system to play the videogame CoastRunners, a plane racing game. Instead of racing toward the finish line, the AI flew circles around an island collecting extra before proceeding. The AI agent scored an average of 20% more points than the human players, however did not carry out the main goal of the videogame itself (competing in the races).',
       'On September 8, 2020, the Guardian published an op-ed generated by OpenAI’s GPT-3 text generator. The editors prompted GPT-3 to write an op-ed on about “why humans have nothing to fear from AI,” but some passages in the resulting output took a threatening tone, including “I know that I will not be able to avoid destroying humankind.” In a note the editors add that they used GPT-3 to generate eight different responses and the human editors splic

In [13]:
cfns.columns

Index(['classifications.Annotator', 'classifications.Annotation Status',
       'classifications.Reviewer', 'classifications.Quality Control',
       'classifications.Full Description', 'classifications.Short Description',
       'classifications.Beginning Date', 'classifications.Ending Date',
       'classifications.Location', 'classifications.Near Miss',
       'classifications.Named Entities', 'classifications.Technology Purveyor',
       'classifications.Intent', 'classifications.Severity',
       'classifications.Lives Lost', 'classifications.Harm Distribution Basis',
       'classifications.Harm Type', 'classifications.Infrastructure Sectors',
       'classifications.Financial Cost', 'classifications.Laws Implicated',
       'classifications.AI System Description', 'classifications.Data Inputs',
       'classifications.System Developer',
       'classifications.Sector of Deployment',
       'classifications.Public Sector Deployment',
       'classifications.Nature of End User',
 

In [16]:
for i in cfns.columns:
    u = cfns[i].unique()
    if len(u) < 10:
        print(i, u)
    else: print(i)

classifications.Annotator [1 2 3 4 6 5]
classifications.Annotation Status ['6. Complete and final' '3. In peer review' '4. Peer review complete']
classifications.Reviewer [5 6 2 3]
classifications.Quality Control [False  True]
classifications.Full Description
classifications.Short Description
classifications.Beginning Date
classifications.Ending Date
classifications.Location
classifications.Near Miss ['Harm caused' 'Unclear/unknown' 'Near miss']
classifications.Named Entities
classifications.Technology Purveyor
classifications.Intent ['Accident' 'Unclear' 'Deliberate or expected']
classifications.Severity ['Moderate' 'Critical' 'Severe' 'Minor' 'Negligible' 'Unclear/unknown']
classifications.Lives Lost [False  True]
classifications.Harm Distribution Basis
classifications.Harm Type
classifications.Infrastructure Sectors
classifications.Financial Cost [nan 'Short term: $1 trillion   unclear of long-term impact' '$528.00'
 '3.7M Ether ($70M at the time)']
classifications.Laws Implicated
c

In [22]:
cfns.head(1)

,classifications.Annotator,classifications.Annotation Status,classifications.Reviewer,classifications.Quality Control,classifications.Full Description,classifications.Short Description,classifications.Beginning Date,classifications.Ending Date,classifications.Location,classifications.Near Miss,...,classifications.Sector of Deployment,classifications.Public Sector Deployment,classifications.Nature of End User,classifications.Level of Autonomy,classifications.Relevant AI functions,classifications.AI Techniques,classifications.AI Applications,classifications.Physical System,classifications.Problem Nature,classifications.Publish
0,1,6. Complete and final,5,False,"On December 5, 2018, a robot punctured a can o...",Twenty-four Amazon workers in New Jersey were ...,2018-12-05T08:00:00.000Z,2018-12-05T08:00:00.000Z,"Robbinsville, NJ",Harm caused,...,Transportation and storage,False,NaN,Unclear/unknown,"[""Unclear""]",NaN,"[""robotics""]","[""Unknown/unclear""]","[""Unknown/unclear""]",True


In [42]:
s = cfns['classifications.Harm Type'].value_counts()
q = np.percentile(s, 50)
s[s > q].index

Index(['[]', '["Harm to social or political systems"]',
       '["Harm to physical health/safety"]', '["Psychological harm"]',
       '["Harm to civil liberties"]', '["Financial harm"]',
       '["Psychological harm","Harm to social or political systems"]',
       '["Harm to physical health/safety","Harm to physical property"]'],
      dtype='object')

In [44]:
s = cfns['classifications.Harm Type'].apply(eval).apply(pd.Series).stack().value_counts()
q = np.percentile(s, 50)
s[s > q].index

Index(['Harm to social or political systems', 'Harm to physical health/safety',
       'Harm to civil liberties', 'Psychological harm', 'Financial harm',
       'Harm to physical property'],
      dtype='object')

In [45]:
s

Harm to social or political systems                           18
Harm to physical health/safety                                17
Harm to civil liberties                                       17
Psychological harm                                            17
Financial harm                                                 9
Harm to physical property                                      8
Harm to intangible property                                    3
Other:Privacy                                                  2
Other:Reputational harm/social harm (libel and defamation)     1
Other:Reputational harm                                        1
Other:Unable to detect facial features                         1
Other:Reputational harm; False incarceration                   1
Other:Harm to publicly available information                   1
dtype: int64

In [46]:
cfns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 33 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   classifications.Annotator                 96 non-null     int64 
 1   classifications.Annotation Status         96 non-null     object
 2   classifications.Reviewer                  96 non-null     int64 
 3   classifications.Quality Control           96 non-null     bool  
 4   classifications.Full Description          96 non-null     object
 5   classifications.Short Description         96 non-null     object
 6   classifications.Beginning Date            95 non-null     object
 7   classifications.Ending Date               94 non-null     object
 8   classifications.Location                  79 non-null     object
 9   classifications.Near Miss                 96 non-null     object
 10  classifications.Named Entities            96 non-nul